In [5]:
get_ipython().system('pip install yfinance')
get_ipython().system('pip install PyPortfolioOpt')
get_ipython().system('pip install cvxopt')
get_ipython().system('pip install cvxpy')
get_ipython().system('pip install matplotlib')
get_ipython().system('pip install sklearn')
import pypfopt
import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pypfopt import risk_models
from pypfopt import plotting
from pypfopt import expected_returns
from pypfopt import EfficientFrontier
from pypfopt import DiscreteAllocation
from pypfopt import objective_functions
from pypfopt import DiscreteAllocation
get_ipython().system('pip install qiskit')
get_ipython().run_line_magic('matplotlib', 'inline')
from qiskit.finance import QiskitFinanceError
from qiskit.finance.data_providers import *
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
import datetime
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

from qiskit import Aer
from qiskit.circuit.library import TwoLocal
from qiskit.aqua import QuantumInstance
from qiskit.finance.applications.ising import portfolio
from qiskit.optimization.applications.ising.common import sample_most_likely
from qiskit.finance.data_providers import RandomDataProvider
from qiskit.aqua.algorithms import VQE, QAOA, NumPyMinimumEigensolver
from qiskit.aqua.components.optimizers import COBYLA
import numpy as np
import matplotlib.pyplot as plt
import datetime

/srv/conda/envs/notebook/lib/python3.7/site-packages/qiskit/finance/__init__.py:50: DeprecationWarning: The package qiskit.finance is deprecated. It was moved/refactored to qiskit_finance (pip install qiskit-finance). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/master/README.md#migration-guide>
  warn_package('finance', 'qiskit_finance', 'qiskit-finance')


In [17]:
def returns(prices):
    mu = expected_returns.capm_return(prices)
    return mu

def min_variance(prices):
    # Long/Short Min Variance Portfolio
    S = risk_models.CovarianceShrinkage(prices).ledoit_wolf()
    ef = EfficientFrontier(None, S, weight_bounds=(None, None))
    ef.min_volatility()
    weights = ef.clean_weights()
    ef.portfolio_performance(verbose=True);
    return(weights)

def Max_Sharpe_port(prices):
    # Max Sharpe Portfolio
    mu = returns(prices)
    S = risk_models.CovarianceShrinkage(prices).ledoit_wolf()
    ef = EfficientFrontier(mu, S) 
    ef.max_sharpe()
    weights = ef.clean_weights()
    ef.portfolio_performance(verbose=True)
    return weights

risk = 0.15
def return_given_risk(prices):
    mu = returns(prices)
    S = risk_models.CovarianceShrinkage(prices).ledoit_wolf()
    ef = EfficientFrontier(mu, S)
    ef.add_objective(objective_functions.L2_reg)  # gamme is the tuning parameter
    ef.efficient_risk(0.15)
    weights = ef.clean_weights()
    ef.portfolio_performance(verbose=True)
    return weights

def minimize_risk(prices):
    mu = returns(prices)
    S = risk_models.CovarianceShrinkage(prices).ledoit_wolf()
    ef = EfficientFrontier(mu, S, weight_bounds=(None, None))
    ef.add_objective(objective_functions.L2_reg)
    ef.efficient_return(target_return=0.07, market_neutral=True)
    weights = ef.clean_weights()
    ef.portfolio_performance(verbose=True);
    return weights

def generate_prices(tickers):
    tickers = tickers 
    ohlc = yf.download(tickers, end="2020-12-23")
    prices = ohlc["Adj Close"].dropna(how="all")
    return(prices)

def index_to_selection(i, num_assets):
    s = "{0:b}".format(i).rjust(num_assets)
    x = np.array([1 if s[i]=='1' else 0 for i in reversed(range(num_assets))])
    return x

def print_result(result):
    selection = sample_most_likely(result.eigenstate)
    value = portfolio.portfolio_value(selection, mu, sigma, q, budget, penalty)
    print('Optimal: selection {}, value {:.4f}'.format(selection, value))

    eigenvector = result.eigenstate if isinstance(result.eigenstate, np.ndarray) else result.eigenstate.to_matrix()
    probabilities = np.abs(eigenvector)**2
    i_sorted = reversed(np.argsort(probabilities))
    print('\n----------------- Full result ---------------------')
    print('selection\tvalue\t\tprobability')
    print('---------------------------------------------------')
    for i in i_sorted:
        x = index_to_selection(i, num_assets)
        value = portfolio.portfolio_value(x, mu, sigma, q, budget, penalty)
        probability = probabilities[i]
        print('%10s\t%.4f\t\t%.4f' %(x, value, probability))

# Discrete Allocation
# latest_prices = prices.iloc[-1]  # prices as of the day you are allocating
# da = DiscreteAllocation(weights, latest_prices, total_portfolio_value=20000, short_ratio=0.3)
# alloc, leftover = da.lp_portfolio()
# print(f"Discrete allocation performed with ${leftover:.2f} leftover")
# alloc

# VQE Backend - Shitty Results , erratic convergence pattern
# backend = Aer.get_backend('statevector_simulator')
# seed = 50

# cobyla = COBYLA()
# cobyla.set_options(maxiter=500)
# ry = TwoLocal(qubitOp.num_qubits, 'ry', 'cz', reps=3, entanglement='full')
# vqe = VQE(qubitOp, ry, cobyla)
# vqe.random_seed = seed

# quantum_instance = QuantumInstance(backend=backend, seed_simulator=seed, seed_transpiler=seed)

# result = vqe.run(quantum_instance)

# print_result(result)

def quantum(tickers):
    data = YahooDataProvider(
                     tickers = tickers,
#                      start=datetime.datetime(2015,3,23),
                     end=datetime.datetime(2020, 3, 23))
    data.run()

    num_assets = 5

    mu = data.get_period_return_mean_vector()
    sigma = data.get_period_return_covariance_matrix()

    q = 0.5                   # set risk factor
    budget = num_assets // 2  # set budget
    penalty = num_assets      # set parameter to scale the budget penalty term

    qubitOp, offset = portfolio.get_operator(mu, sigma, q, budget, penalty)

    backend = Aer.get_backend('statevector_simulator')
    seed = 50

    cobyla = COBYLA()
    cobyla.set_options(maxiter=250)
    qaoa = QAOA(qubitOp, cobyla, 3)

    qaoa.random_seed = seed

    quantum_instance = QuantumInstance(backend=backend, seed_simulator=seed, seed_transpiler=seed)

    result = qaoa.run(quantum_instance)

    print_result(result)

In [ ]:
# sorted by top AUM

tech = ['QQQ', 'VGT', 'XLK', 'ARKK', 'FDN'] 
healthcare = ['XLV', 'VHT','IBB','ARKG','IHI']
financials = ['XLF', 'VFH', 'KRE', 'KBE', 'FAS']
real_estate = ['VNQ', 'SCHH', 'VNQI', 'IYR', 'REET']
energy = ['XLE', 'ICLN', 'AMLP', 'VDE', 'XOP']
materials = ['GDX', 'XLB', 'GDXJ', 'GUNR', 'VAW']
consumer = ['XLY', 'XLC', 'VCR', 'PEJ', 'FXD']
industrials = ['XLI', 'VIS', 'JETS', 'ITA', 'IYT']
telecom = ['VOX', 'NXTG', 'FCOM', 'IYZ', 'IXP']
consumer_staples = ['XLP', 'VDC', 'IEV', 'FSTA', 'IYK']
utilities = ['XLU', 'VPU', 'IGF', 'NFRA', 'PAVE']

all_industries = ["MSFT", "AMZN", "KO", "MA", "COST",
           "LUV", "XOM", "PFE", "JPM", "UNH", 
           "ACN", "DIS", "GILD", "F", "TSLA"]

## Test 1: Cross-Industry Portfolios

In [7]:
# Test 1 

# Classical Portfolio
test1 = ["MSFT", "AMZN", "KO", "MA", "COST"]
prices = generate_prices(test1)
# cov = risk_models.CovarianceShrinkage(prices).ledoit_wolf()
# plotting.plot_covariance(cov, plot_correlation=True)
Max_Sharpe_port(prices)

[*********************100%***********************]  5 of 5 completed
Expected annual return: 23.2%
Annual volatility: 15.1%
Sharpe Ratio: 1.41


OrderedDict([('AMZN', 0.20923),
             ('COST', 0.11975),
             ('KO', 0.11957),
             ('MA', 0.46735),
             ('MSFT', 0.0841)])

In [26]:
# Quantum Portfolio

quantum(test1)

Optimal: selection [0. 0. 1. 0. 1.], value -0.0006

----------------- Full result ---------------------
selection	value		probability
---------------------------------------------------
[0 0 1 0 1]	-0.0006		0.0892
[0 0 1 1 0]	-0.0003		0.0892
[1 0 1 0 0]	0.0002		0.0891
[0 1 1 0 0]	0.0007		0.0890
[0 0 0 1 1]	-0.0008		0.0889
[1 0 0 0 1]	-0.0003		0.0888
[1 0 0 1 0]	0.0000		0.0888
[0 1 0 0 1]	0.0002		0.0888
[0 1 0 1 0]	0.0004		0.0887
[1 1 0 0 0]	0.0010		0.0887
[0 0 0 0 0]	20.0000		0.0112
[0 0 1 1 1]	4.9993		0.0069
[1 0 1 1 0]	5.0002		0.0069
[1 0 1 0 1]	4.9999		0.0069
[0 1 1 0 1]	5.0003		0.0069
[0 1 1 1 0]	5.0006		0.0069
[1 1 1 0 0]	5.0011		0.0069
[1 1 0 1 0]	5.0009		0.0068
[1 0 0 1 1]	4.9997		0.0068
[0 1 0 1 1]	5.0001		0.0068
[1 1 0 0 1]	5.0006		0.0068
[1 1 1 1 1]	45.0010		0.0043
[0 0 1 0 0]	4.9999		0.0040
[0 0 0 0 1]	4.9994		0.0040
[0 0 0 1 0]	4.9996		0.0040
[1 0 0 0 0]	5.0002		0.0040
[0 1 0 0 0]	5.0006		0.0040
[1 0 1 1 1]	19.9999		0.0014
[1 1 1 1 0]	20.0011		0.0014
[0 1 1 1 1]	20.0003		0.0

In [8]:
prices_2 = generate_prices(['MSFT','AMZN'])
Max_Sharpe_port(prices_2)

[*********************100%***********************]  2 of 2 completed
Expected annual return: 40.6%
Annual volatility: 34.8%
Sharpe Ratio: 1.11


OrderedDict([('AMZN', 0.56604), ('MSFT', 0.43396)])

# Test 1.B

In [27]:
test_2 = ["LUV", "XOM", "PFE", "JPM", "UNH"]
prices = generate_prices(test_2)
# cov = risk_models.CovarianceShrinkage(prices).ledoit_wolf()
# plotting.plot_covariance(cov, plot_correlation=True)
Max_Sharpe_port(prices)

[*********************100%***********************]  5 of 5 completed
Expected annual return: 16.7%
Annual volatility: 18.4%
Sharpe Ratio: 0.80


OrderedDict([('JPM', 0.20965),
             ('LUV', 0.20761),
             ('PFE', 0.19906),
             ('UNH', 0.25321),
             ('XOM', 0.13047)])

In [33]:
quantum(test_2)

Optimal: selection [0. 0. 0. 1. 1.], value -0.0008

----------------- Full result ---------------------
selection	value		probability
---------------------------------------------------
[0 0 0 1 1]	-0.0008		0.0891
[0 0 1 0 1]	-0.0006		0.0890
[0 0 1 1 0]	-0.0003		0.0889
[1 0 0 0 1]	-0.0003		0.0889
[1 0 0 1 0]	0.0000		0.0888
[0 1 0 0 1]	0.0002		0.0887
[1 0 1 0 0]	0.0002		0.0887
[0 1 0 1 0]	0.0004		0.0886
[0 1 1 0 0]	0.0007		0.0885
[1 1 0 0 0]	0.0010		0.0884
[0 0 0 0 0]	20.0000		0.0182
[0 0 0 0 1]	4.9994		0.0109
[0 0 0 1 0]	4.9996		0.0109
[0 0 1 0 0]	4.9999		0.0108
[1 0 0 0 0]	5.0002		0.0108
[0 1 0 0 0]	5.0006		0.0107
[1 0 1 1 1]	19.9999		0.0030
[1 1 0 1 1]	20.0007		0.0030
[0 1 1 1 1]	20.0003		0.0030
[1 1 1 1 0]	20.0011		0.0030
[1 1 1 0 1]	20.0008		0.0030
[0 1 1 0 1]	5.0003		0.0025
[1 0 1 0 1]	4.9999		0.0025
[1 1 0 1 0]	5.0009		0.0025
[1 1 1 0 0]	5.0011		0.0025
[1 0 1 1 0]	5.0002		0.0025
[0 1 0 1 1]	5.0001		0.0025
[0 1 1 1 0]	5.0006		0.0025
[1 0 0 1 1]	4.9997		0.0025
[0 0 1 1 1]	4.9993		0.

In [37]:
prices = generate_prices(['PFE', 'UNH', "XOM"])
Max_Sharpe_port(prices)

[*********************100%***********************]  3 of 3 completed
Expected annual return: 17.7%
Annual volatility: 19.7%
Sharpe Ratio: 0.79


OrderedDict([('PFE', 0.32759), ('UNH', 0.43179), ('XOM', 0.24062)])

# Test 1.C 

In [30]:
# Test 3 

test_3 = ["ACN", "DIS", "GILD", "F", "TSLA"]
prices = generate_prices(test_3)
cov = risk_models.CovarianceShrinkage(prices).ledoit_wolf()
# plotting.plot_covariance(cov, plot_correlation=True)

Max_Sharpe_port(prices)

[*********************100%***********************]  5 of 5 completed
Expected annual return: 18.9%
Annual volatility: 14.5%
Sharpe Ratio: 1.17


OrderedDict([('ACN', 0.27102),
             ('DIS', 0.08611),
             ('F', 0.05369),
             ('GILD', 0.16673),
             ('TSLA', 0.42245)])

In [31]:
quantum(test_3)

Optimal: selection [1. 0. 0. 0. 1.], value -0.0003

----------------- Full result ---------------------
selection	value		probability
---------------------------------------------------
[1 0 0 0 1]	-0.0003		0.0902
[0 1 0 0 1]	0.0002		0.0902
[0 0 1 0 1]	-0.0006		0.0902
[1 1 0 0 0]	0.0010		0.0902
[1 0 1 0 0]	0.0002		0.0902
[0 0 0 1 1]	-0.0008		0.0901
[0 1 1 0 0]	0.0007		0.0901
[1 0 0 1 0]	0.0000		0.0901
[0 1 0 1 0]	0.0004		0.0901
[0 0 1 1 0]	-0.0003		0.0901
[0 0 0 0 0]	20.0000		0.0092
[1 1 1 0 1]	20.0008		0.0087
[1 0 1 1 1]	19.9999		0.0087
[1 1 1 1 0]	20.0011		0.0087
[1 1 0 1 1]	20.0007		0.0087
[0 1 1 1 1]	20.0003		0.0087
[1 0 1 0 1]	4.9999		0.0041
[1 1 0 0 1]	5.0006		0.0041
[0 1 1 0 1]	5.0003		0.0041
[1 1 1 0 0]	5.0011		0.0041
[1 0 0 1 1]	4.9997		0.0041
[1 0 1 1 0]	5.0002		0.0041
[0 0 1 1 1]	4.9993		0.0041
[1 1 0 1 0]	5.0009		0.0041
[0 1 0 1 1]	5.0001		0.0041
[0 1 1 1 0]	5.0006		0.0041
[1 1 1 1 1]	45.0010		0.0022
[0 0 0 0 1]	4.9994		0.0006
[1 0 0 0 0]	5.0002		0.0006
[0 0 1 0 0]	4.9999		0

In [32]:
prices = generate_prices(['ACN', 'TSLA'])
Max_Sharpe_port(prices)

[*********************100%***********************]  2 of 2 completed
Expected annual return: 22.9%
Annual volatility: 27.3%
Sharpe Ratio: 0.77


OrderedDict([('ACN', 0.43082), ('TSLA', 0.56918)])

# Test 2 - Targeted Portfolios

In [59]:
COVID= ["ZM", "PFE", "NFLX", "AAPL", "GRUB"]
prices = generate_prices(COVID)
# cov = risk_models.CovarianceShrinkage(prices).ledoit_wolf()
# plotting.plot_covariance(cov, plot_correlation=True)
Max_Sharpe_port(prices)

[*********************100%***********************]  5 of 5 completed
Expected annual return: 27.3%
Annual volatility: 11.1%
Sharpe Ratio: 2.28


OrderedDict([('AAPL', 0.03878),
             ('GRUB', 0.20191),
             ('NFLX', 0.06737),
             ('PFE', 0.04823),
             ('ZM', 0.64371)])

In [48]:
quantum(COVID)

Optimal: selection [1. 1. 0. 0. 1.], value 5.0006

----------------- Full result ---------------------
selection	value		probability
---------------------------------------------------
[1 1 0 0 1]	5.0006		0.0833
[0 0 1 1 0]	-0.0003		0.0833
[0 1 0 0 1]	0.0002		0.0833
[1 0 1 1 0]	5.0002		0.0833
[0 0 0 1 1]	-0.0008		0.0833
[0 1 1 0 0]	0.0007		0.0833
[1 1 1 0 0]	5.0011		0.0833
[1 0 0 1 1]	4.9997		0.0833
[0 1 0 1 0]	0.0004		0.0833
[1 1 0 1 0]	5.0009		0.0833
[0 0 1 0 1]	-0.0006		0.0833
[1 0 1 0 1]	4.9999		0.0833
[0 0 0 0 0]	20.0000		0.0000
[1 0 0 0 0]	5.0002		0.0000
[0 1 1 1 1]	20.0003		0.0000
[1 1 1 1 1]	45.0010		0.0000
[1 1 0 1 1]	20.0007		0.0000
[0 1 0 1 1]	5.0001		0.0000
[1 0 1 0 0]	0.0002		0.0000
[0 0 1 0 0]	4.9999		0.0000
[0 1 1 1 0]	5.0006		0.0000
[0 0 0 0 1]	4.9994		0.0000
[1 1 1 1 0]	20.0011		0.0000
[1 0 0 0 1]	-0.0003		0.0000
[1 1 1 0 1]	20.0008		0.0000
[0 1 1 0 1]	5.0003		0.0000
[1 0 0 1 0]	0.0000		0.0000
[0 0 0 1 0]	4.9996		0.0000
[1 0 1 1 1]	19.9999		0.0000
[0 0 1 1 1]	4.9993		0.

In [58]:
prices = generate_prices(["GRUB", 'ZM'])
Max_Sharpe_port(prices)

[*********************100%***********************]  2 of 2 completed
Expected annual return: 36.3%
Annual volatility: 32.3%
Sharpe Ratio: 1.07


OrderedDict([('GRUB', 0.32045), ('ZM', 0.67955)])

# Test 2.B

In [51]:
# Portfolio 2, go Long when hope is shown. Decemeber 14
post_COVID = ['UBER', 'AAL', 'HLT', 'CMG', 'PYPL']
prices = generate_prices(post_COVID)
# cov = risk_models.CovarianceShrinkage(prices).ledoit_wolf()
# plotting.plot_covariance(cov, plot_correlation=True)
Max_Sharpe_port(prices)

[*********************100%***********************]  5 of 5 completed
Expected annual return: 29.0%
Annual volatility: 21.1%
Sharpe Ratio: 1.28


OrderedDict([('AAL', 0.07581),
             ('CMG', 0.01817),
             ('HLT', 0.0),
             ('PYPL', 0.0),
             ('UBER', 0.90602)])

In [56]:
data = YahooDataProvider(tickers = post_COVID,
                     start=datetime.datetime(2019,6,15),
                     end=datetime.datetime(2020, 12, 23))
data.run()

num_assets = 5

mu = data.get_period_return_mean_vector()
sigma = data.get_period_return_covariance_matrix()
q = 0.5                   # set risk factor
budget = num_assets // 2  # set budget
penalty = num_assets      # set parameter to scale the budget penalty term

qubitOp, offset = portfolio.get_operator(mu, sigma, q, budget, penalty)

backend = Aer.get_backend('statevector_simulator')
seed = 50

cobyla = COBYLA()
cobyla.set_options(maxiter=250)
qaoa = QAOA(qubitOp, cobyla, 3)

qaoa.random_seed = seed

quantum_instance = QuantumInstance(backend=backend, seed_simulator=seed, seed_transpiler=seed)

result = qaoa.run(quantum_instance)

print_result(result)

Optimal: selection [0. 1. 1. 0. 0.], value 0.0026

----------------- Full result ---------------------
selection	value		probability
---------------------------------------------------
[0 1 1 0 0]	0.0026		0.0743
[1 1 0 0 0]	0.0022		0.0742
[0 1 0 1 0]	0.0006		0.0738
[0 1 0 0 1]	0.0004		0.0738
[1 0 1 0 0]	-0.0002		0.0737
[1 0 0 1 0]	-0.0016		0.0734
[0 0 1 1 0]	-0.0017		0.0734
[1 0 0 0 1]	-0.0019		0.0733
[0 0 1 0 1]	-0.0019		0.0733
[0 0 0 1 1]	-0.0032		0.0731
[0 1 0 0 0]	5.0020		0.0214
[0 0 1 0 0]	4.9997		0.0212
[1 0 0 0 0]	4.9995		0.0211
[0 0 0 1 0]	4.9983		0.0210
[0 0 0 0 1]	4.9981		0.0210
[1 1 1 0 0]	5.0034		0.0141
[1 1 0 1 0]	5.0014		0.0140
[0 1 1 1 0]	5.0015		0.0140
[1 1 0 0 1]	5.0011		0.0140
[0 1 1 0 1]	5.0012		0.0140
[1 0 1 1 0]	4.9991		0.0139
[0 1 0 1 1]	4.9994		0.0139
[1 0 1 0 1]	4.9987		0.0139
[1 0 0 1 1]	4.9974		0.0138
[0 0 1 1 1]	4.9971		0.0138
[1 1 1 1 1]	45.0026		0.0087
[0 0 0 0 0]	20.0000		0.0043
[1 0 1 1 1]	19.9984		0.0012
[0 1 1 1 1]	20.0006		0.0011
[1 1 0 1 1]	20.0008		0.

In [57]:
post_COVID2 = ['UBER', 'CMG']
prices = generate_prices(post_COVID2)
Max_Sharpe_port(prices)

[*********************100%***********************]  2 of 2 completed
Expected annual return: 41.0%
Annual volatility: 21.3%
Sharpe Ratio: 1.83


OrderedDict([('CMG', 0.06532), ('UBER', 0.93468)])

In [ ]:
get_ipython().system('pip install alpaca-trade-api')

import alpaca_trade_api as tradeapi
# from alpaca_trade_api.rest import TimeFrame
# import pandas as pd

APCA_API_KEY_ID = ''
APCA_API_SECRET_KEY = ''
APCA_API_BASE_URL = 'https://paper-api.alpaca.markets'

# Setting up API and account details
api = tradeapi.REST(
    APCA_API_KEY_ID,
    APCA_API_SECRET_KEY,
    base_url=APCA_API_BASE_URL,
    api_version='v2')

def market_order_from_list(orderList):
    """
    Creates live market buy orders which expire at end of the day from a list of tuples in the form:
    orderList = [('AAPL', '10'), ('JPM', '5')]
    """
    for ticker, amount in orderList:
        api.submit_order(
            symbol=f'{ticker}',
            side='buy',
            type='market',
            qty=f'{amount}',
            # TODO: Do some calculation to get max amount under some percentage of current cash
            time_in_force='day'
        )

market_order_from_list(orderList)

api.list_orders()
api.cancel_all_orders()
